In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.utils import compare_car_speeds, compute_track_dominance_multi, quali_track_evolution
from src.plotset import setup_plot, save_fig, plot_track_dominance, plot_quali_track_evolution

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2024,15,'Q')
session.load()

In [ ]:
points, segments, colors = compute_track_dominance_multi(session=session,drivers=['NOR','PIA'],circuit_length=4259,colors_map={'NOR':'#FF8000','PIA':'#FFFFFF'},window_size=100)

In [ ]:
fig = plot_track_dominance(points=points, segments=segments, colors=colors,figsize=(12,11))

In [ ]:
save_fig(fig=fig, name='track_dominance', loc='Reel14')

In [ ]:
df = quali_track_evolution(session=session)

In [ ]:
round(df.Q2.mean() - df.Q1.mean(),1), round(df.Q3.mean() - df.Q2.mean(),1)

In [ ]:
setup_plot(xyticksize=22,axeslabel=24,figtitle=30)
fig = plot_quali_track_evolution(df)

In [ ]:
save_fig(fig=fig, name='quali_evolution', loc='Reel14')

In [ ]:
corner_inputs = {'low':[12],
                 'med':[8],
                 'high':[7]}

car_perf = compare_car_speeds(session=session, drivers=['NOR','PIA'],corner_inputs=corner_inputs,delta=10)
car_perf

In [ ]:
q2024 = session.results[['Q1','Q2','Q3']].min(axis=1).dt.total_seconds()
q2025 = q2024 + pd.Series(np.random.randn(20),index=q2024.index)

In [ ]:
q_delta = pd.concat([q2024,q2025],axis=1)

In [ ]:
q_delta

In [ ]:
dabb = [session.get_driver(i).Abbreviation for i in q2024.index]

q_delta.index = dabb
q_delta.columns = ['2024','2025']

In [ ]:
q_delta

In [ ]:
q_delta['delta'] = q_delta['2025'] - q_delta['2024']

In [ ]:
team_delta = pd.Series([])

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.barplot(data=q_delta,x=q_delta.index,y='delta',hue=q_delta.index,palette=plotting.get_driver_color_mapping(session=session))

ax.set_xlabel('')
ax.set_ylabel('Lap Time Delta (s)')
ax.set_title('Lap Time Improvement vs 2024 (-ve is better)')